## Reconstruction Gallery

In [ ]:
# TODO - move any reasonable-to-precomput torch files to disk, put unreasonable ones in gitignore
#      - add params for gallery size, flag for force recompute, and cache loading for prerenders

import torch
import numpy as np
from rendering import find_best_angle_from_part, render_part, render_mesh
from tqdm.notebook import tqdm
from render_shape import preds_to_mesh
from zipfile import ZipFile
import json
from pspy import Part
from train_latent_space import BRepFaceAutoencoder
from matplotlib import pyplot as plt
import os

data_root = '/media/ben/Data/cad'
f360seg_index_path =  data_root + '/fusion360seg.json'
f360seg_zip_path = data_root + '/fusion360seg.zip'
model_checkpoint_path = data_root + '/BRepFaceAutoencoder_64_1024_4.ckpt'
computed_f360seg_codes_path = data_root + '/fusion360seg_coded.pt'
render_losses_path = 'f360_render_test_losses.pt'
figure_out_path = 'renderings.png'

rows = 6
cols = 4
size = 5

grid_density = 100

force_rendering = False





testing_losses = torch.load(render_losses_path)
avg_losses = np.array([x[0].item() for x in testing_losses])
part_sizes = np.array([x[1] for x in testing_losses])
total_losses = avg_losses * part_sizes
avg_sorted = sorted(enumerate(zip(avg_losses, part_sizes, total_losses)),key=lambda x: x[1][0])
total_sorted = sorted(enumerate(zip(avg_losses, part_sizes, total_losses)),key=lambda x: x[1][2])
total_filtered = [x for x in total_sorted if x[1][1] > 20]


recompute_renderings = True
if os.path.exists('render_results.pt') and not force_rendering:
    render_results = torch.load('render_results.pt')
    renders = render_results['renders']
    gts = render_results['gts']
    if len(renders) >= rows*cols:
        recompute_renderings = False

if recompute_renderings:
    with open(f360seg_index_path,'r') as f:
        index = json.load(f)
    data =  ZipFile(f360seg_zip_path,'r')
    parts_list = [index['template'].format(*x) for x in index['test']]

    model = BRepFaceAutoencoder(64,1024,4)
    ckpt = torch.load(model_checkpoint_path)
    model.load_state_dict(ckpt['state_dict'])

    def predict(face_codes, model, N=grid_density):
        n_faces = face_codes.shape[0]
        line = torch.linspace(-0.1,1.1,N)
        grid = torch.cartesian_prod(line, line)
        grids = grid.repeat(n_faces,1)
        indices = torch.arange(n_faces).repeat_interleave(N*N, dim=0)
        with torch.no_grad():
            indexed_codes = face_codes[indices]
            uv_codes = torch.cat([grids, indexed_codes],dim=1)
            preds = model.decoder(uv_codes)
        return preds

    codes = torch.load(computed_f360seg_codes_path)

    gts = []
    renders = []
    for k in tqdm(range(rows*cols)):
        i = total_filtered[k][0]
        N = grid_density
        preds = predict(codes[parts_list[i]]['x'], model, N)
        V, F, C = preds_to_mesh(preds, N)
        part = Part(data.open(parts_list[i]).read().decode('utf-8'))
        pose, zoom = find_best_angle_from_part(part)
        ground_truth = render_part(part, pose, zoom)
        rendering = render_mesh(V,F,C,pose,zoom)
        gts.append(ground_truth)
        renders.append(rendering)

    render_results = {'gts':gts,'renders':renders}
    torch.save(render_results, 'render_results.pt')



M = rows*cols
s = size
fig, axes = plt.subplots(int(M/cols), cols*2, figsize=(2*cols*s, s*int(M/cols)),gridspec_kw = {'wspace':0, 'hspace':0}, dpi=300)
for i in range(M):
    row = int(i / cols)
    col = i % cols
    axes[row,2*col].imshow(gts[i])
    axes[row,2*col].axis('off')
    axes[row,2*col+1].imshow(renders[i])
    axes[row,2*col+1].axis('off')
fig.savefig(figure_out_path)

## MFCAD Figure

In [ ]:
from precoded_training import DictDatamodule, CodePredictor, load_models
from pylab import cm
import meshplot as mp

with open('/projects/grail/benjonesnb/cadlab/siggasia2022/mfcad.json','r') as f:
    index = json.load(f)
    coded_set = torch.load('/projects/grail/benjonesnb/cadlab/siggasia2022/precoded/mfcad_coded.pt')

datamodule = DictDatamodule(
                    index, 
                    coded_set,  
                    seed=0, 
                    batch_size=512
                )

test_ds = datamodule.ds_test

all_models = load_models('/projects/grail/benjonesnb/cadlab/siggasia2022/tensorboard/mfcad/')
our_models = all_models['mp2']

# Compute a "difficulty" scale for test set examples (this is also used in comparison figs...)
accs = np.zeros((len(test_ds), len(our_models)))
for i in tqdm(range(len(test_ds))):
    data = test_ds[i]
    for j, (ts, m) in enumerate(our_models):
        with torch.no_grad():
            preds = m(data).argmax(dim=1).numpy()
        targets = data.y.numpy()
        acc = (preds.flatten() == targets.flatten()).sum() / len(targets.flatten())
        accs[i,j] = acc

# Setup Plotting Code (Replace with Other Version)
num_labels = max([i for j in index['test_labels'] for i in j]) + 1
cmap = cm.get_cmap('tab20', num_labels)
zf = ZipFile('/projects/grail/benjonesnb/cadlab/siggasia2022/mfcad.zip')
test_keys = [index['template'].format(*x) for x in index['test']]
def plot_part(V, F, E2T, FC=None):
    plot = mp.plot(V, F, c = FC)
    E = np.concatenate([
        F[E2T[:,0]>=0][:,[2,0]], 
        F[E2T[:,1]>=0][:,[0,1]],
        F[E2T[:,2]>=0][:,[1,2]]
    ], axis=0)
    plot.add_edges(V, E, shading={'line_width':0.5})
def plot_preds(i, test_ds, test_keys=test_keys, zf=zf, models=our_models, colormap=cmap, just_gt=False):
    data = test_ds[i]
    with zf.open(test_keys[i],'r') as f:
        p = Part(f.read().decode('utf-8'))
    target = data.y.numpy()
    target_c = colormap(target[p.mesh_topology.face_to_topology])[:,:3]
    plot_part(p.mesh.V, p.mesh.F, p.mesh_topology.edge_to_topology,  target_c)
    if just_gt:
        return
    model_preds = []
    for ts, model in models:
        with torch.no_grad():
            preds = model(data).argmax(dim=1).numpy()
            model_preds.append(preds)   
    for preds in model_preds:
        pred_c = colormap(preds[p.mesh_topology.face_to_topology])[:,:3]
        plot_part(p.mesh.V, p.mesh.F, p.mesh_topology.edge_to_topology, pred_c)


# Metrics used to help find the examples
local_auc = accs.sum(axis = 1) - accs[:,0] / 2 - accs[:,-1] / 2
complexity = np.array([len(test_ds[i].y) for i in range(len(test_ds))])
sorted_by_auc = sorted(enumerate(zip(local_auc, complexity)), key=lambda x:x[1][0])
filtered_examples = [x for x in sorted_by_auc if x[1][1] > 20]

gallery_examples = [646, 965, 673, 417, 888, 1090]

for i in gallery_examples:
    plot_preds(i, test_ds, test_keys, zf, our_models, color_pallet, just_gt=True)


## Limitations Plot

In [ ]:
import altair as alt
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

accs = pd.read_parquet('df_face_acc.parquet')
accs['one'] = 1
iscorr = np.vectorize(lambda x: 'correct' if x == 1 else 'incorrect')
accs['correct'] = iscorr(accs.label)
accs['logmse'] = np.log(accs.mse)

alt.data_transformers.disable_max_rows()

alt.Chart(accs[accs.mse < 1]).mark_bar().encode(
    x=alt.X('logmse',bin=alt.Bin(maxbins=50)),
    y=alt.Y('sum(one)', stack='normalize', axis=alt.Axis(format='%')),
    color=alt.Color('label:N')
).facet(row='train_size')

## Classification Comparisons

In [ ]:
import numpy as np
import torch
import json
from precoded_training import DictDatamodule, CodePredictor, load_models
from rendering import find_best_angle_from_part, render_part2
from tqdm.notebook import tqdm
from pspy import Part
from zipfile import ZipFile
from matplotlib import pyplot as plt


# Setup Local Paths

repbrep = '/home/ben/Documents/research/repbrep/'
model_dir = os.path.join(repbrep, 'models', 'BRepFaceAutoencoder_64_1024_4')
checkpoint_path = os.path.join(model_dir, 'BRepFaceAutoencoder_64_1024_4.ckpt')

f360seg_index_path =  os.path.join(repbrep, 'datasets', 'fusion360seg.json')
f360seg_zip_path = os.path.join(repbrep, 'datasets', 'fusion360seg.zip')
f360seg_tb_dir = os.path.join(repbrep, 'training_logs', 'ours', 'f360seg')
f360seg_coded_path = os.path.join(model_dir, 'fusion360seg_coded.pt')

mfcad_index_path =  os.path.join(repbrep, 'datasets', 'mfcad.json')
mfcad_zip_path = os.path.join(repbrep, 'datasets', 'mfcad.zip')
mfcad_tb_dir = os.path.join(repbrep, 'training_logs', 'ours', 'mfcad')
mfcad_coded_path = os.path.join(model_dir, 'mfcad_coded.pt')

f360seg_uvnet_results_path = os.path.join(repbrep, 'results', 'Fusion360Seg_UV-Net.json')
f360seg_brepnet_results_path = os.path.join(repbrep, 'results', 'Fusion360Seg_BRepNet.json')

#data_root = '/media/ben/Data/cad'
#f360seg_index_path =  data_root + '/fusion360seg.json'
#f360seg_zip_path = data_root + '/fusion360seg.zip'

#repbrep_data_path = '/home/ben/Documents/research/repbrep/data'

#f360_seg_coded_path = repbrep_data_path + '/precoded/fusion360seg_coded.pt'

#f360_seg_uvnet_results_path = repbrep_data_path + '/baselines/Fusion360Seg_UV-Net.json'
#f360seg_brepnet_results_path = repbrep_data_path + '/baselines/Fusion360Seg_BRepNet.json'

#experiments_dir = '/home/ben/Documents/research/repbrep/data/training/experiments'
#f360seg_tb_dir = experiments_dir +'/f360seg/'



# Load UV-Net Results
with open(f360seg_uvnet_results_path,'r') as f:
    uvnet360 = json.load(f)

labels = uvnet360['labels']
logits = uvnet360['logits']['100'][0]
uv_preds = [np.array(l).argmax() for l in logits]
part_accs_uv = []
offset = 0
part_preds_uv = []
for part_labels in labels:
    part_preds = uv_preds[offset:offset+len(part_labels)]
    part_preds_uv.append(part_preds)
    part_accs_uv.append((np.array(part_preds) == np.array(part_labels)).sum() / len(part_labels))
    offset += len(part_labels)
part_accs_uv = np.array(part_accs_uv)

# Load BRepNet Results
with open(f360seg_brepnet_results_path,'r') as f:
    brep360 = json.load(f)

labels = brep360['labels']
logits = brep360['logits']['100'][0]
preds = [np.array(l).argmax() for ll in logits for l in ll]
part_accs_brep = []
part_preds_brep = []
offset = 0
for i, part_labels in enumerate(labels):
    part_preds = preds[offset:offset+len(part_labels)]
    part_preds_brep.append(part_preds)
    part_accs_brep.append((np.array(part_preds) == np.array(part_labels)).sum() / len(part_labels))
    offset += len(part_labels)
part_accs_brep = np.array(part_accs_brep)


# Load Our Trained Models
all_models = load_models(f360seg_tb_dir)
our_models = all_models['mp2']

# Select Our Model
model = our_models[1][1]

# Load Fusion 360 Dataset
with open(f360seg_index_path,'r') as f:
    index = json.load(f)
    coded_set = torch.load(f360seg_coded_path)



datamodule = DictDatamodule(
                    index, 
                    coded_set, 
                    train_size=23266, 
                    seed=0, 
                    batch_size=512
                )

test_ds = datamodule.ds_test


# Compute Our Model's Accuracy at different train sizes
accs = np.zeros((len(test_ds), len(our_models)))
for i in tqdm(range(len(test_ds))):
    data = test_ds[i]
    for j, (ts, m) in enumerate(our_models):
        with torch.no_grad():
            preds = m(data).argmax(dim=1).numpy()
        targets = data.y.numpy()
        acc = (preds.flatten() == targets.flatten()).sum() / len(targets.flatten())
        accs[i,j] = acc

# We just take the accuracies for the size we used
part_accs_ours = accs[:,1]


# Compute the total accuracy lift over baselines for each test example
lift = (part_accs_ours - part_accs_brep) + (part_accs_ours - part_accs_uv)

# Compute the number of faces in each test example as a complexity metric
complexity = np.array([len(test_ds[i].y) for i in range(len(test_ds))])

# Sort by lift and other keys, and filter to parts with more than 25 faces, and baselines with at least 10% accuracy
nuanced_wins = [x for x in sorted(enumerate(zip(lift,complexity, part_accs_ours, part_accs_uv, part_accs_brep)), reverse=True, key = lambda x : x[1]) 
 if x[1][1] > 25 and x[1][3] > .1 and x[1][4] > .1]

# Here is where we could try selecting random models.


data =  ZipFile(f360seg_zip_path,'r')
parts_list = [index['template'].format(*x) for x in index['test']]

s=5
M = 10

selection = nuanced_wins[:M]
fig, axes = plt.subplots(M,4, figsize=(4*s,M*s),gridspec_kw={'wspace':0,'hspace':0},dpi=300)
for k, sel in enumerate(tqdm(selection)):
    i = sel[0]
    part = Part(data.open(parts_list[i]).read().decode('utf-8'))
    pose, zoom = find_best_angle_from_part(part, cubify=True)

    gt = np.array(labels[i])
    pdata = test_ds[i]
    our_preds = model(pdata).argmax(dim=1).numpy().astype(int)
    uv_preds = np.array(part_preds_uv[i]).astype(int)
    brep_preds = np.array(part_preds_brep[i]).astype(int)

    gt_im = render_part2(part, pose, zoom, face_labels=gt, max_labels=8)
    our_im = render_part2(part, pose, zoom, face_labels=our_preds, max_labels=8)
    uv_im = render_part2(part, pose, zoom, face_labels=uv_preds, max_labels=8)
    brep_im = render_part2(part, pose, zoom, face_labels=brep_preds, max_labels=8)
    axes[k,0].imshow(gt_im)
    axes[k,1].imshow(our_im)
    axes[k,2].imshow(uv_im)
    axes[k,3].imshow(brep_im)
    for ax in axes[k]:
        ax.axis('off')

In [18]:
pdata = test_ds[i]
our_preds = model(pdata)
our_preds.shape

torch.Size([26, 16])

In [213]:
mfcad_tb_dir = experiments_dir +'/f360seg/'
our_models_all_seeds_f360 = [load_models(f360seg_tb_dir, s)['mp2'] for s in range(10)]
our_models_all_seeds_mfcad = [load_models(mfcad_tb_dir, s)['mp2'] for s in range(10)]

In [ ]:
our_models_all_seeds_f360[0]

In [ ]:
import os
import json
import torch
from tqdm.notebook import tqdm
from precoded_training import DictDatamodule, CodePredictor, load_models

repbrep = '/home/ben/Documents/research/repbrep/'
model_dir = os.path.join(repbrep, 'models', 'BRepFaceAutoencoder_64_1024_4')
checkpoint_path = os.path.join(model_dir, 'BRepFaceAutoencoder_64_1024_4.ckpt')

f360seg_index_path =  os.path.join(repbrep, 'datasets', 'fusion360seg.json')
f360seg_zip_path = os.path.join(repbrep, 'datasets', 'fusion360seg.zip')
f360seg_tb_dir = os.path.join(repbrep, 'training_logs', 'ours', 'f360seg')
f360seg_coded_path = os.path.join(model_dir, 'fusion360seg_coded.pt')

mfcad_index_path =  os.path.join(repbrep, 'datasets', 'mfcad.json')
mfcad_zip_path = os.path.join(repbrep, 'datasets', 'mfcad.zip')
mfcad_tb_dir = os.path.join(repbrep, 'training_logs', 'ours', 'mfcad')
mfcad_coded_path = os.path.join(model_dir, 'mfcad_coded.pt')


our_models_all_seeds_f360 = [load_models(f360seg_tb_dir, s)['mp2'] for s in range(10)]
our_models_all_seeds_mfcad = [load_models(mfcad_tb_dir, s)['mp2'] for s in range(10)]


def load_testset(index_path, coded_path):
    with open(index_path, 'r') as f:
        index = json.load(f)
    coded_set = torch.load(coded_path)
    datamodule = DictDatamodule(
        index,
        coded_set,
        train_size=1000,
        seed=0,
        batch_size=100000
    )
    test_ds = datamodule.ds_test
    return test_ds

f360_test = load_testset(f360seg_index_path, f360seg_coded_path)
mfcad_test = load_testset(mfcad_index_path, mfcad_coded_path)


labels = []
records = []
for seed in tqdm(range(10), 'Seed'):
    for model_name, model_set, test_ds in tqdm([
        ('Fusion360Seg', our_models_all_seeds_f360, f360_test), 
        ('MFCAD', our_models_all_seeds_mfcad, mfcad_test)], 'dataset', leave=False):
        for train_size, model in tqdm(model_set[seed], 'train_size', leave=False):
            for i in tqdm(range(len(test_ds)), 'examples', leave=False):
                data = test_ds[i]
                with torch.no_grad():
                    pred_logits = model(data).numpy().tolist()
                iter_labels = data.y.numpy().tolist()
                records.append({
                    'dataset': model_name,
                    'model': 'Ours',
                    'test_idx': i,
                    'seed': seed,
                    'train_size': train_size,
                    'logits': pred_logits,
                    'labels': iter_labels
                })

In [7]:
import pandas as pd

our_df = pd.DataFrame.from_records(records)

In [257]:
our_df.to_parquet('results/OursAll.parquet')

In [21]:
import numpy as np

In [22]:
np.argmax([0,5,6])

2

In [23]:
pred_maxes = [np.argmax(face_logits) 
    for part_logits in our_df[our_df.dataset == 'Fusion360Seg'].logits.values 
    for face_logits in part_logits
]

In [27]:
from matplotlib import pyplot as plt

In [39]:
(np.array(pred_maxes) == 4).sum()

175297

In [9]:
all_logits.append
for part_logits in our_df.logits:
    logits = [logit[:8] for logit in part_logits]

,dataset,model,test_idx,seed,train_size,logits,labels
0,Fusion360Seg,Ours,0,0,10,"[[2.6588728427886963, 2.5666146278381348, 1.73...","[6, 6, 6, 6, 6, 6, 6]"
1,Fusion360Seg,Ours,1,0,10,"[[2.9058258533477783, 0.6674148440361023, 1.87...","[5, 1, 0, 5, 0, 1, 6, 6, 6]"
2,Fusion360Seg,Ours,2,0,10,"[[2.6696524620056152, -0.32663166522979736, 3....","[0, 1, 0, 1, 0]"
3,Fusion360Seg,Ours,3,0,10,"[[4.333590984344482, 2.064574718475342, 3.3654...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]"
4,Fusion360Seg,Ours,4,0,10,"[[5.578789710998535, 3.209151268005371, 3.9268...","[0, 0, 0, 0, 1, 1]"
...,...,...,...,...,...,...,...
328435,MFCAD,Ours,1543,9,13940,"[[-28.5069637298584, -28.329050064086914, -10....","[15, 15, 15, 15, 15, 3, 3, 3, 3, 3, 3, 15, 3, ..."
328436,MFCAD,Ours,1544,9,13940,"[[-4.1662726402282715, -21.894088745117188, -8...","[15, 15, 5, 5, 5, 15, 15, 15, 15, 3, 3, 3, 3, ..."
328437,MFCAD,Ours,1545,9,13940,"[[-27.35886573791504, -32.08065414428711, -6.8...","[15, 15, 7, 7, 15, 15, 15, 3, 3, 3, 3, 3, 3, 7..."
328438,MFCAD,Ours,1546,9,13940,"[[-12.003915786743164, -43.92551040649414, -15...","[15, 15, 5, 15, 8, 15, 5, 5, 8, 15, 15, 15, 0,..."


In [40]:
our_df.to_parquet('our_results.parquet')

In [ ]:
# Compute Our Model's Accuracy at different train sizes
accs = np.zeros((len(test_ds), len(our_models)))
for i in tqdm(range(len(test_ds))):
    data = test_ds[i]
    for j, (ts, m) in enumerate(our_models):
        with torch.no_grad():
            preds = m(data).argmax(dim=1).numpy()
        targets = data.y.numpy()
        acc = (preds.flatten() == targets.flatten()).sum() / len(targets.flatten())
        accs[i,j] = acc

# We just take the accuracies for the size we used
part_accs_ours = accs[:,1]

In [11]:
import json
import numpy as np
from tqdm.notebook import tqdm

data_root = '/media/ben/Data/cad'
f360seg_index_path =  data_root + '/fusion360seg.json'
f360seg_zip_path = data_root + '/fusion360seg.zip'

repbrep_data_path = '/home/ben/Documents/research/repbrep/data'

f360_seg_coded_path = repbrep_data_path + '/precoded/fusion360seg_coded.pt'

f360seg_uvnet_results_path = repbrep_data_path + '/baselines/Fusion360Seg_UV-Net.json'
f360seg_brepnet_results_path = repbrep_data_path + '/baselines/Fusion360Seg_BRepNet.json'

mfcad_uvnet_results_path = repbrep_data_path + '/baselines/MFCAD_UV-Net.json'
mfcad_brepnet_results_path = repbrep_data_path + '/baselines/MFCAD_BRepNet.json'

experiments_dir = '/home/ben/Documents/research/repbrep/data/training/experiments'
f360seg_tb_dir = experiments_dir +'/f360seg/'

def load_json(path):
    with open(path,'r') as f:
        return json.load(f)

def load_uvnet_results(uvnet360, train_size, seed):
    labels = uvnet360['labels']
    logits = uvnet360['logits'][str(train_size)][seed]
    uv_preds = [np.array(l).argmax() for l in logits]
    part_accs_uv = []
    offset = 0
    part_preds_uv = []
    for part_labels in labels:
        part_preds = uv_preds[offset:offset+len(part_labels)]
        part_preds_uv.append(part_preds)
        part_accs_uv.append((np.array(part_preds) == np.array(part_labels)).sum() / len(part_labels))
        offset += len(part_labels)
    part_accs_uv = np.array(part_accs_uv)
    return part_preds_uv, part_accs_uv

def load_all_uv_net_results(uvnet360, seed):
    sizes = [int(x) for x in uvnet360['logits'].keys()]
    results = {}
    for s in sizes:
        preds, accs = load_uvnet_results(uvnet360, s, seed)
        results[s] = {'preds':preds,'accs':accs}
    return results


def load_brepnet_results(brep360, size, seed):
    labels = brep360['labels']
    logits = brep360['logits'][str(size)][seed]
    preds = [np.array(l).argmax() for ll in logits for l in ll]
    part_accs_brep = []
    part_preds_brep = []
    offset = 0
    for i, part_labels in enumerate(labels):
        part_preds = preds[offset:offset+len(part_labels)]
        part_preds_brep.append(part_preds)
        part_accs_brep.append((np.array(part_preds) == np.array(part_labels)).sum() / len(part_labels))
        offset += len(part_labels)
    part_accs_brep = np.array(part_accs_brep)
    return part_preds_brep, part_accs_brep

#res = load_all_uv_net_results(uvnet360, 0)

In [12]:
uvnet360 = load_json(f360seg_uvnet_results_path)
uvnet_mf = load_json(mfcad_uvnet_results_path)

brep360 = load_json(f360seg_brepnet_results_path)
brep_mf = load_json(mfcad_brepnet_results_path)

In [13]:
uvnet360.keys()

dict_keys(['dataset', 'model', 'checkpoint', 'labels', 'logits'])

In [58]:
uvnet_mf.keys()

dict_keys(['dataset', 'model', 'checkpoint', 'labels', 'logits'])

In [120]:
uvnet_fab = load_json('/home/ben/Documents/research/repbrep/data/baselines/FabWave_UV-Net.json')

In [121]:
uvnet_fab.keys()

dict_keys(['dataset', 'model', 'checkpoint', 'labels', 'logits', 'train_samples', 'test_omitted_samples'])

In [118]:
len(brep360['logits']['10'][0][4][5])

8

In [151]:
uvnet_fab['test_omitted_samples']

{'1': [92, 92, 92, 92, 92, 92, 92, 92, 92, 92],
 '5': [92, 92, 92, 92, 92, 92, 92, 92, 92, 92],
 '10': [92, 92, 92, 92, 92, 92, 92, 92, 92, 92],
 '20': [92, 92, 92, 92, 92, 92, 92, 92, 92, 92],
 '50': [92, 92, 92, 92, 92, 92, 92, 92, 92, 92],
 '75': [92, 92, 92, 92, 92, 92, 92, 92, 92, 92],
 '100': [92, 92, 92, 92, 92, 92, 92, 92, 92, 92]}

In [150]:
# UV-Net FabWave Format:
# 'dataset': str ('FabWave')
# 'model': str ('UV-Net')
# 'checkpoint': dict
#    str(train_size (as pct)): list(str) (absolute path to checkpoint)
# 'labels': dict str -> list(list(  int))
#                 |      |     |     |
#           train pct   seed  part  part type label
# 'logits': dict str -> list(list(list(float)))
#                 |      |     |   \_______/
#             train pct seed  part  logits(26)
# 'train_samples': dict str -> list(int)
#                        |      |    |
#                   train pct  seed  #train samples
# 'test_omitted_samples': dict str -> list(int)
#                               |      |    |
#                          train pct  seed  #ommited test samples


# BRepNet Fusion360Seg Format:
# 'dataset': str ('Fusion360Seg')
# 'model': str ('BRepNet')
# 'checkpoint': dict
#    str(train_size): list(str) (RELATIVE path to checkpoint)
# 'labels': list(list(int)) (one list for each part, int labels for faces in part)
# 'logits': dict
#    str(train_size): list(list(list(list(float))) (first level is seed, second is face (so part-face has been flattened), third is length 8 floats)
#                      |     |    |  \________/
#                  seeds  parts faces  logits(8)

# BRepNet MFCAD Format:
# 'dataset': str ('MFCAD')
# 'model': str ('BRepNet')
# 'checkpoint': dict
#    str(train_size): list(str) (RELATIVE path to checkpoint)
# 'labels': list(list(int)) (one list for each part, int labels for faces in part)
# 'logits': dict
#    str(train_size): list(list(list(list(float))) (first level is seed, second is face (so part-face has been flattened), third is length 8 floats)
#                      |     |    |  \--------/
#                  seeds  parts faces  logits(16)


# UV-Net MFCAD Format:
# 'dataset': str ('MFCAD')
# 'model': str ('UV-Net')
# 'checkpoint': dict
#    str(train_size): list(str) (ABSOLUTE path to checkpoint)
# 'labels': list(list(int)) (one list for each part, int labels for faces in part)
# 'logits': dict
#    str(train_size): list(list(list(float))) (first level is seed, second is face (so part-face has been flattened), third is length 8 floats)
#                      |     |  \--------/
#                    seeds  faces logits

# UV-Net Fusion 360 Format:
# 'dataset': str ('Fusion360Seg')
# 'model': str ('UV-Net')
# 'checkpoint': dict
#    str(train_size): list(str) (ABSOLUTE path to checkpoint)
# 'labels': list(list(int)) (one list for each part, int labels for faces in part)
# 'logits': dict
#    str(train_size): list(list(list(float))) (first level is seed, second is face (so part-face has been flattened), third is length 8 floats)
#                      |     |  \--------/
#                    seeds  faces logits

In [184]:
def convert_uvnet(original):
    results = {}
    results['dataset'] = original['dataset']
    results['model'] = original['model']
    results['checkpoint'] = {
        int(k) : list(map(lambda x: x[29:], v)) 
        for k,v in original['checkpoint'].items()
        }
    results['labels'] = original['labels']
    logits = {int(k):v for k,v in original['logits'].items()}
    part_sizes = list(map(len, results['labels']))
    results['logits'] = {}
    for k,v in logits.items():
        seeds = []
        for seed_logits in v:
            curr_logits = []
            offset = 0
            for part_size in part_sizes:
                curr_logits.append(seed_logits[offset:offset+part_size])
                offset = offset + part_size
            seeds.append(curr_logits)
        results['logits'][k] = seeds
    return results

def convert_brep(original):
    results = {}
    results['dataset'] = original['dataset']
    results['model'] = original['model']
    results['checkpoint'] = {int(k):v for k,v in original['checkpoint'].items()}
    results['labels'] = original['labels']
    results['logits'] = {int(k):v for k,v in original['logits'].items()}
    return results

c_uvnet360 = convert_uvnet(uvnet360)
c_uvnet_mf = convert_uvnet(uvnet_mf)
c_brep360 = convert_brep(brep360)
c_brep_mf = convert_brep(brep_mf)

In [187]:
os.path.dirname('models/test_file.obj')

'models'

In [190]:
def write_json(obj, filename):
    directory = os.path.dirname(filename)
    os.makedirs(directory, exist_ok=True)
    with open(filename, 'w') as f:
        json.dump(obj, f)

In [194]:
#write_json(c_uvnet360, 'results/Fusion360Seg_UV-Net.json')
#write_json(c_uvnet_mf, 'results/MFCAD_UV-Net.json')
write_json(c_brep360, 'results/Fusion360Seg_BRepNet.json')
write_json(c_brep_mf, 'results/MFCAD_BRepNet.json')

In [192]:
s = json.dumps(c_uvnet360)

In [193]:
# Parqet-Style Results
# Model, Dataset, TestIdx, Train Size, Seed, Test Model, Accuracy, Logits, Preds, Labels, Checkpoint

524165731

In [44]:
our_df.iloc[0]

dataset                                            Fusion360Seg
model                                                      Ours
test_idx                                                      0
seed                                                          0
train_size                                                   10
logits        [[2.6588728427886963, 2.5666146278381348, 1.73...
labels                                    [6, 6, 6, 6, 6, 6, 6]
Name: 0, dtype: object

In [84]:
per_face_recs = []

for curr_df in [our_df, baselines_df]:
    for i in tqdm(range(len(curr_df))):
        rec = curr_df.iloc[i]
        for face_idx, logit in enumerate(rec.logits):
            if rec.dataset == 'Fusion360Seg':
                logit = logit[:8]
            logit = np.array(logit)
            prediction = np.argmax(logit)
            target = rec.labels[face_idx]
            correct = (prediction == target)
            per_face_recs.append({
                'dataset': rec.dataset,
                'model': rec.model,
                'test_idx': rec.test_idx,
                'face_idx': face_idx,
                'seed': rec.seed,
                'train_size': rec.train_size,
                'scores': logit,
                'prediction': prediction,
                'target': target,
                'correct': correct
            })

per_face_df = pd.DataFrame.from_records(per_face_recs)

  0%|          | 0/328440 [00:00<?, ?it/s]

  0%|          | 0/656880 [00:00<?, ?it/s]

In [46]:
baselines_df = pd.read_parquet('/home/ben/Documents/research/repbrep/results/all_baselines.parquet')

In [54]:
np.argmax(baselines_df[baselines_df.dataset == 'Fusion360Seg'].logits[0][0])

0

In [88]:
per_face_df.to_parquet('/home/ben/Documents/research/repbrep/results/ours_and_baselines_per_face.parquet')

In [90]:
per_face_df[['dataset','model','train_size','correct']].groupby(['dataset','model','train_size']).agg(np.mean)

correct
dataset      model   train_size          
Fusion360Seg BRepNet 10          0.527208
                     100         0.537144
                     1000        0.569395
                     10000       0.872465
                     20000       0.900546
                     23266       0.902462
             Ours    10          0.564120
                     100         0.675405
                     1000        0.830958
                     10000       0.919429
                     20000       0.935691
                     23266       0.938257
             UV-Net  10          0.341749
                     100         0.454888
                     1000        0.761662
                     10000       0.901075
                     20000       0.922596
                     23266       0.924702
MFCAD        BRepNet 10          0.249180
                     100         0.414782
                     1000        0.920018
                     10000       0.994175
                     13940       0.999209
             Ours    10          0.377878
                     100         0.652727
                     1000        0.948662
                     10000       0.991476
                     13940       0.993930
             UV-Net  10          0.284335
                     100         0.432883
                     1000        0.984418
                     10000       0.999627
                     13940       0.999778

In [87]:
per_face_df.model.value_counts()

Ours       4931760
UV-Net     4931760
BRepNet    4931760
Name: model, dtype: int64

In [69]:
f360seg_index_path

'/home/ben/Documents/research/repbrep/datasets/fusion360seg.json'

In [70]:
with open(mfcad_index_path, 'r') as f:
    mfcad_index = json.load(f)
with open(f360seg_index_path, 'r') as f:
    f360seg_index = json.load(f)
with open('/home/ben/Documents/research/repbrep/datasets/fabwave.json','r') as f:
    fabwave_index = json.load(f)

In [75]:
len(fabwave_index['train'])

2613

In [79]:
baselines_df.model.value_counts()

UV-Net     328440
BRepNet    328440
Name: model, dtype: int64

In [80]:
our_df.model.value_counts()

Ours    328440
Name: model, dtype: int64

In [82]:
baselines_df.columns

Index(['dataset', 'model', 'test_idx', 'seed', 'train_size', 'logits',
       'labels', 'checkpoint'],
      dtype='object')

In [83]:
our_df.columns

Index(['dataset', 'model', 'test_idx', 'seed', 'train_size', 'logits',
       'labels'],
      dtype='object')

In [ ]:
# Precompute Camera Angles

from rendering import find_best_angle_from_part
from tqdm.notebook import tqdm
from zipfile import ZipFile
import json
import os
from pspy import Part, PartOptions




def get_camera_params(index_path, zip_path):
    poses = []
    zooms = []
    opts = PartOptions()
    opts.default_mcfs = False
    opts.num_uv_samples = 0
    opts.sample_normals = 0
    opts.sample_tangents = False
    with open(index_path, 'r') as f:
        index = json.load(f)
    parts_list = [index['template'].format(*x) for x in index['test']]
    with ZipFile(zip_path, 'r') as zf:
        for part_path in tqdm(parts_list):
            part = Part(zf.open(part_path).read().decode('utf-8'), opts)
            pose, zoom = find_best_angle_from_part(part, cubify=True)
            poses.append(pose)
            zooms.append(zoom)
    return poses, zooms

repbrep = '/home/ben/Documents/research/repbrep/'
model_dir = os.path.join(repbrep, 'models', 'BRepFaceAutoencoder_64_1024_4')
checkpoint_path = os.path.join(model_dir, 'BRepFaceAutoencoder_64_1024_4.ckpt')

f360seg_index_path =  os.path.join(repbrep, 'datasets', 'fusion360seg.json')
f360seg_zip_path = os.path.join(repbrep, 'datasets', 'fusion360seg.zip')
f360seg_tb_dir = os.path.join(repbrep, 'training_logs', 'ours', 'f360seg')
f360seg_coded_path = os.path.join(model_dir, 'fusion360seg_coded.pt')

mfcad_index_path =  os.path.join(repbrep, 'datasets', 'mfcad.json')
mfcad_zip_path = os.path.join(repbrep, 'datasets', 'mfcad.zip')
mfcad_tb_dir = os.path.join(repbrep, 'training_logs', 'ours', 'mfcad')
mfcad_coded_path = os.path.join(model_dir, 'mfcad_coded.pt')

f360_poses, f360_zooms = get_camera_params(mfcad_index_path, mfcad_zip_path)
mfcad_poses, mfcad_zooms = get_camera_params(mfcad_index_path, mfcad_zip_path)